# Heston Almost Exact Simulation

This is a qablet adaptation of [Heston Almost Exact Simulation](https://github.com/nburgessx/Papers/tree/main/HestonSimulation) by Nicholas Burgess.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from qablet.base.mc import MCModel, MCStateBase
from numpy.random import Generator, SFC64
from qablet.base.utils import Forwards
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.eq.forward import ForwardOption
from qablet_contracts.eq.cliquet import Accumulator
from qablet_contracts.timetable import py_to_ts
from src.qablet_utils import option_prices

## Create the Model State Class

In [ ]:
def CIR_Sample(NoOfPaths, kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 * kappa * vbar / gamma / gamma
    c = 1.0 / (4.0 * kappa) * gamma * gamma * (1.0 - np.exp(-kappa * (t - s)))
    kappaBar = (
        4.0
        * kappa
        * v_s
        * np.exp(-kappa * (t - s))
        / (gamma * gamma * (1.0 - np.exp(-kappa * (t - s))))
    )
    sample = c * np.random.noncentral_chisquare(delta, kappaBar, NoOfPaths)
    return sample


class HestonAESMCState(MCStateBase):
    def __init__(self, timetable, dataset):
        super().__init__(timetable, dataset)

        # fetch the model parameters from the dataset
        self.n = dataset["MC"]["PATHS"]
        self.asset = dataset["HESTON"]["ASSET"]
        self.asset_fwd = Forwards(dataset["ASSETS"][self.asset])
        self.spot = self.asset_fwd.forward(0)

        self.gamma = dataset["HESTON"]["VOL_OF_VAR"]
        self.kappa = dataset["HESTON"]["MEANREV"]
        self.vbar = dataset["HESTON"]["LONG_VAR"]
        self.rho = dataset["HESTON"]["CORRELATION"]

        # Initialize the arrays
        self.rng = Generator(SFC64(dataset["MC"]["SEED"]))
        self.x_vec = np.zeros(self.n)  # process x (log stock)
        # Initialize as a scalar, it will become a vector in the advance method
        self.v = dataset["HESTON"]["INITIAL_VAR"]

        self.cur_time = 0

    def advance(self, new_time):
        """Update x_vec in place when we move simulation by time dt."""

        dt = new_time - self.cur_time
        if dt < 1e-10:
            return

        r = self.asset_fwd.rate(new_time, self.cur_time)

        # Generate the Brownian Increments
        dw_vec = self.rng.standard_normal(self.n) * np.sqrt(dt)  # * self.vol

        # Exact samples for the variance process
        new_v = CIR_Sample(
            self.n, self.kappa, self.gamma, self.vbar, 0, dt, self.v
        )

        # AES Constant Terms
        k0 = (r - self.rho / self.gamma * self.kappa * self.vbar) * dt
        k1 = (
            self.rho * self.kappa / self.gamma - 0.5
        ) * dt - self.rho / self.gamma
        k2 = self.rho / self.gamma

        # Almost Exact Simulation for Log-Normal Asset Process
        self.x_vec += (
            k0
            + k1 * self.v
            + k2 * new_v
            + np.sqrt((1.0 - self.rho**2) * self.v) * dw_vec
        )

        self.v = new_v
        self.cur_time = new_time

    def get_value(self, unit):
        """Return the value of the unit at the current time.
        This model uses black scholes model for one asset, return its value using the simulated array.
        For any other asset that may exist in the timetable, just return the default implementation in
        the model base (i.e. simply return the forwards)."""

        if unit == self.asset:
            return self.spot * np.exp(self.x_vec)
        else:
            return None

## Create the Model class
We will now create the model class. In this case all we have to do is specify the state_class to be used by this model.

In [ ]:
class HestonAESMC(MCModel):
    def state_class(self):
        return HestonAESMCState

## Create Dataset
Create the dataset, with MC params, discounts and fwds as in previous examples. Add the parameters needed by our model.

In [ ]:
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

ticker = "EQ"
spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))

dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(datetime(2023, 12, 31)).value,
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {
        "PATHS": 2_500,
        "TIMESTEP": 1 / 1000,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VAR": 1.0,
        "MEANREV": 0.5,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Price

In [ ]:
model = HestonAESMC()

# Create Contract
strike = 100
ticker = "EQ"
timetable = Option(
    "USD", ticker, strike=strike, maturity=datetime(2024, 12, 31), is_call=True
).timetable()
print(timetable["events"].to_pandas())

price, stats = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

  track                      time op  quantity unit
0       2024-12-31 00:00:00+00:00  >       0.0  USD
1       2024-12-31 00:00:00+00:00  +    -100.0  USD
2       2024-12-31 00:00:00+00:00  +       1.0   EQ
price:   12.259944


## Generate Multiple Option prices

In [ ]:
expirations = [
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 12, 31),
]
strikes = np.array([0.8, 0.9, 1.0, 1.1, 1.2]) * spot
is_call = True
prices = option_prices(ticker, expirations, strikes, is_call, model, dataset)
print(prices)

   Strike  2024-03-31  2024-06-30  2024-12-31
0    80.0   22.369317   24.738565   28.790613
1    90.0   13.260894   15.932704   20.433361
2   100.0    4.885113    7.561712   12.291547
3   110.0    0.138555    0.880307    4.709466
4   120.0    0.003813    0.039283    0.412092


## Forward Starting Option

In [ ]:
for strike_date in [
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 11, 30),
]:
    timetable = ForwardOption(
        "USD",
        ticker,
        strike_rate=1.0,
        strike_date=strike_date,
        maturity=datetime(2024, 12, 31),
        is_call=True,
    ).timetable()
    price, stats = model.price(timetable, dataset)
    print(f"strike_date: {strike_date.date()} price: {price:11.6f}")

strike_date: 2024-03-31 price:    9.311379
strike_date: 2024-06-30 price:    6.251180
strike_date: 2024-11-30 price:    1.448775


## Accumulator Cliquet

In [ ]:
# Create the cliquet, quarterly fixing dates
fix_dates = pd.bdate_range(
    datetime(2023, 12, 31), datetime(2024, 12, 31), freq="1BQE"
)

for local_cap in [0.02, 0.04, 0.06]:
    global_floor = 0.0
    local_floor = -local_cap
    timetable = Accumulator(
        "USD",
        ticker,
        fix_dates,
        global_floor,
        local_floor,
        local_cap,
        state={"S_PREV": 1.0},
    ).timetable()

    price, stats = model.price(timetable, dataset)
    print(f"cap/floor: {local_cap} price: {price:11.6f}")

price:    9.252146
